In [1]:
import tensorflow as tf
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('always.csv', index_col=0)

In [3]:
df.head()

,close,log_ret
2022-02-03 00:00:00,36902.96,NaN
2022-02-03 00:00:01,36902.81,-0.000004
2022-02-03 00:00:02,36899.19,-0.000098
2022-02-03 00:00:03,36900.54,0.000037
2022-02-03 00:00:04,36895.62,-0.000133


In [13]:
class WindowGenerator():
    def __init__(self, data, input_width, label_width, shift, splitpct=(0.7, 0.9),
               label_columns=None):
        # Store the raw data (DF).
        self.train_df = data[:int(splitpct[0] * len(data))]
        self.val_df = data[int(splitpct[0] * len(data)):int(splitpct[1] * len(data))]
        self.test_df = data[int(splitpct[1] * len(data)):]

        self.train_ds = self.make_dataset(self.train_df)
        self.val_ds = self.make_dataset(self.val_df)
        self.test_ds = self.make_dataset(self.test_df)

        # Work out the label column indices.
        self.label_columns = label_columns
        if label_columns is not None:
          self.label_columns_indices = {name: i for i, name in
                                        enumerate(label_columns)}
        self.column_indices = {name: i for i, name in
                               enumerate(train_df.columns)}

        # Work out the window parameters.
        self.input_width = input_width
        self.label_width = label_width
        self.shift = shift

        self.total_window_size = input_width + shift
        self.input_slice = slice(0, input_width)
        self.input_indices = np.arange(self.total_window_size)[self.input_slice]

        self.label_start = self.total_window_size - self.label_width
        self.labels_slice = slice(self.label_start, None)
        self.label_indices = np.arange(self.total_window_size)[self.labels_slice]

    # I DONT GET
    def split_window(self, features):
        inputs = features[:, self.input_slice, :]
        labels = features[:, self.labels_slice, :]
        if self.label_columns is not None:
            labels = tf.stack([
                labels[:, :, self.column_indices[name]]
                for name in self.label_columns
            ],
                              axis=-1)
    
    # --> tf.DataSet (or smt equivalent)
    def make_dataset(self, data):
        data = np.array(data, dtype=np.float32)
        ds = tf.keras.utils.timeseries_dataset_from_array(
            data=data,
            targets=None,
            sequence_length=self.total_window_size,
            sequence_stride=1,
            shuffle=True,
            batch_size=32,
        )

        ds = ds.map(self.split_window)

        return ds

    def __repr__(self):
        return '\n'.join([
            f'Total window size: {self.total_window_size}',
            f'Input indices: {self.input_indices}',
            f'Label indices: {self.label_indices}',
            f'Label column name(s): {self.label_columns}'])

In [11]:
ds = tf.keras.utils.timeseries_dataset_from_array(
            data=df,
            targets=None,
            sequence_length=30,
            sequence_stride=1,
            shuffle=True,
            batch_size=32,
        )

Metal device set to: Apple M1 Pro


2022-02-09 16:59:55.544093: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:305] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2022-02-09 16:59:55.544321: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:271] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [10]:
np.arange(31)[slice(29, None)]

array([29, 30])

In [9]:
slice(29, None)

slice(29, None, None)

In [ ]:
conv_window = WindowGenerator(
    input_width=30, #look at past 30 seconds
    label_width=10, #do it for the next 10 seconds
    shift=1, #predict at next or shifth time step
    label_columns=['T (degC)'])

conv_window

In [ ]:
def stationize(series):
    return np.log(series / series.shift(1))